In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("NLP").getOrCreate()

In [5]:
from pyspark.ml.feature import HashingTF,IDF, Tokenizer

In [6]:
sentenceData= spark.createDataFrame([
    (0.0,"Hi I heard about Spark"),
    (0.0,"I wish java could use case classes"),
    (1.0,"Logistic regression models are neat")
],["label","sentence"])

In [7]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol="sentence", outputCol = "words")

In [ ]:
# first tokenization then getting term frequency and inverse document frequency 

In [9]:
words_data = tokenizer.transform(sentenceData)

In [10]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, ı, heard, ab...|
|  0.0|I wish java could...|[ı, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [11]:
hashing_tf = HashingTF(inputCol="words", outputCol = "rawFeatures")

In [12]:
featurized_data = hashing_tf.transform(words_data)

In [13]:
idf = IDF(inputCol = "rawFeatures", outputCol = "features")

In [14]:
idf_model= idf.fit(featurized_data)

In [15]:
rescaled_data = idf_model.transform(featurized_data)

In [17]:
rescaled_data.select("label","features").show(truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                       |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,33808,66273,173558,231588],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.28768207245178085])                                                   |
|0.0  |(262144,[20719,55551,58672,98717,109547,192310,231588],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.28768207245178085])|


In [ ]:
# CountVectorizer helps convert a collection of text documents into vectors of word counts

In [19]:
from pyspark.ml.feature import CountVectorizer

In [20]:
df = spark.createDataFrame([
    (0,"a b c".split(" ")),
    (1,"a b b c a".split(" "))
    
],["id","words"])

In [21]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [22]:
cv = CountVectorizer(inputCol = "words", outputCol = "features",vocabSize = 3, minDF = 2.0)  

In [23]:
# minDF affects fitting process by specifying the minimum number of documents,
# a term must appear in to be included in the vocabulary

In [24]:
model = cv.fit(df)

In [25]:
result = model.transform(df)

In [26]:
result.show(truncate = False)## bag of words count

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

